# Amazon Redshift Console Lab
Region: **us-east-1**

This notebook is a **lab guide** for performing an end-to-end Redshift Serverless workflow **entirely through the AWS Management Console** 

Follow each step using the AWS Console and the built-in **Query Editor v2**. You can use this notebook as a **step-by-step manual** to record SQL queries and notes.

---

## Step 1: Create Namespace and Workgroup
1. Log in to the **AWS Management Console**.
2. Navigate to **Amazon Redshift → Redshift Serverless**.
3. Click **Create workgroup** or **Get started** if setting up for the first time.
4. You can choose **Default settings** for simplicity.
   - Namespace: `redshift-serverless-lab`
   - Workgroup: `redshift-workgroup-lab`
   - Database: `dev`
5. Wait for both Namespace and Workgroup to become active.

📘 *A namespace defines data storage and permissions; a workgroup defines compute resources.*

## Step 2: Attach IAM Role and Configure S3 Access
To load or unload data between S3 and Redshift Serverless:

1. Go to **Redshift Serverless Console → Namespaces → [Your Namespace]**.
2. Under **Security and Encryption**, click **Manage IAM roles**.
3. Attach an IAM role that has permissions like:
   - `AmazonS3ReadOnlyAccess` (for loading data)
   - `AmazonS3FullAccess` (if you plan to unload data)
4. Save the role association.

✅ *This enables COPY and UNLOAD commands to work with your S3 buckets.*

## Step 3: Open Query Editor v2
1. In the Redshift Serverless console, click **Query data**.
2. This opens **Query Editor v2**.
3. Choose your **workgroup** to connect.
4. Select your authentication method (user/password or IAM).
5. Once connected, you will see a SQL editor interface to run queries directly in Redshift Serverless.

## Step 4: Create Table (Tickit Data Example)
Run the following SQL statements in **Query Editor v2** to create a table and load sample Tickit data from S3:

```sql
CREATE TABLE IF NOT EXISTS tickit_sales (
  salesid INTEGER NOT NULL,
  listid INTEGER NOT NULL,
  sellerid INTEGER NOT NULL,
  buyerid INTEGER NOT NULL,
  eventid INTEGER NOT NULL,
  dateid SMALLINT NOT NULL,
  qtysold SMALLINT NOT NULL,
  pricepaid DECIMAL(8,2),
  commission DECIMAL(8,2),
  saletime TIMESTAMP
);
```

Now load data from the AWS public Tickit dataset:

```sql
COPY tickit_sales
FROM 's3://awssampledbuswest2/tickit/spectrum/sales/'
IAM_ROLE 'arn:aws:iam::123456789012:role/YourRedshiftRole'
FORMAT AS PARQUET;
```

📘 *The COPY command imports data from S3 into Redshift using the IAM role.*

## Step 5: Run Analytical Queries
After loading the data, you can execute queries to analyze it.

### Total Number of Sales
```sql
SELECT COUNT(*) AS total_sales FROM tickit_sales;
```

### Top 5 Events by Total Revenue
```sql
SELECT eventid, SUM(pricepaid) AS total_revenue
FROM tickit_sales
GROUP BY eventid
ORDER BY total_revenue DESC
LIMIT 5;
```

### Average Ticket Price by Seller
```sql
SELECT sellerid, AVG(pricepaid) AS avg_price
FROM tickit_sales
GROUP BY sellerid
ORDER BY avg_price DESC
LIMIT 10;
```

## Step 6: Unload Query Results Back to S3
You can export query results to S3 using the `UNLOAD` command.

```sql
UNLOAD ('SELECT * FROM tickit_sales WHERE qtysold > 5')
TO 's3://your-output-bucket/tickit_unload/'
IAM_ROLE 'arn:aws:iam::123456789012:role/YourRedshiftRole'
PARQUET
ALLOWOVERWRITE;
```

✅ *This writes the query results in Parquet format back to your S3 bucket.*

## Step 7: Cleanup Resources
Once your lab is complete, clean up to avoid unnecessary costs.

```sql
DROP TABLE IF EXISTS tickit_sales;
```

You can also delete your Redshift **workgroup** and **namespace** from the AWS Console to stop serverless billing.

## ✅ Summary
- This lab used  **AWS Console** and **Query Editor v2**.
- You learned how to:
  1. Create Redshift Serverless resources
  2. Attach IAM roles for S3 access
  3. Load and query Tickit data
  4. Unload data back to S3
  5. Clean up resources safely

---